In [1]:
import os
import cv2
import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
directory = 'input/leapGestRecog/leapGestRecog/00/'
folders = [f for f in os.listdir(directory) if not f.startswith('.')]
lookup = {folder: idx for idx, folder in enumerate(folders)}
reverselookup = {idx: folder for idx, folder in enumerate(folders)}
# print(lookup)

In [3]:
x_data = []
y_data = []
IMG_SIZE = 150
num_classes = 10

datacount = 0 # We'll use this to tally how many images are in our dataset
for i in range(0, 10): # Loop over the ten top-level folders
    for j in os.listdir('input/leapGestRecog/leapGestRecog/0' + str(i) + '/'):
        if not j.startswith('.'):
            count = 0 
            for k in os.listdir('input/leapGestRecog/leapGestRecog/0' + str(i) + '/' + j + '/'):
                path = 'input/leapGestRecog/leapGestRecog/0' + str(i) + '/' + j + '/' + k
                img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
                img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
                arr = np.array(img)
                x_data.append(arr) 
                count = count + 1
            y_values = np.full((count, 1), lookup[j]) 
            y_data.append(y_values)
            datacount = datacount + count
x_data = np.array(x_data, dtype = 'float32')
y_data = np.array(y_data)
y_data = y_data.reshape(datacount, 1) 
y_data=to_categorical(y_data)
x_data = x_data.reshape((datacount, IMG_SIZE, IMG_SIZE, 1))
x_data = x_data/255

x_train,x_test,y_train,y_test=train_test_split(x_data,y_data,test_size=0.30,random_state=42)

In [4]:
x_data = []
y_data = []
IMG_SIZE = 150
num_classes = 10
datacount = 0 
for i in range(0, num_classes):
    for j in os.listdir('input/leapGestRecog/0' + str(i) + '/'):
        if not j.startswith('.'):
            count = 0
            for k in os.listdir('input/leapGestRecog/0' + str(i) + '/' + j + '/'):
                path = 'input/leapGestRecog/0' + str(i) + '/' + j + '/' + k
                img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
                img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
                arr = np.array(img)
                x_data.append(arr) 
                count = count + 1
            y_values = np.full((count, 1), lookup[j]) 
            y_data.append(y_values)
            datacount = datacount + count
x_data = np.array(x_data, dtype = 'float32')
y_data = np.array(y_data)
y_data = y_data.reshape(datacount, 1) # Reshape to be the correct size
y_data=to_categorical(y_data)
x_data = x_data.reshape((datacount, IMG_SIZE, IMG_SIZE, 1))
x_data = x_data/255
x_train,x_test,y_train,y_test=train_test_split(x_data,y_data,test_size=0.30,random_state=42)

In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(5, 5), activation='relu', input_shape=(IMG_SIZE,IMG_SIZE,1)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Summary of the model
model.summary()

# Train model 
# Training parameters
batch_size = 8
epochs = 3

history = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=epochs, batch_size=batch_size)

# Evaluate the model
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f'Test accuracy: {test_acc}')

In [ ]:
# Visualize some sample frames
def visualize_samples(X, y, num_samples=5):
    for i in range(num_samples):
        plt.figure(figsize=(10, 2))
        for j in range(10):  # show first 10 frames
            plt.subplot(1, 10, j+1)
            plt.imshow(X[i][j])
            plt.axis('off')
        plt.title(f'Class: {np.argmax(y[i])}')
        plt.show()

print('Visualizing samples')
visualize_samples(x_train, y_train)